In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/LICA

/content/drive/.shortcut-targets-by-id/1o9bmmzSkE9nR90YsEAsPAonsHgVaRQwP/LICA


In [ ]:
import os
import time
import numpy as np
import cv2

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from utils import load_data
#from alexnet import AlexNet
from model_uj import Model

In [ ]:
# define pytorch device - useful for device-agnostic execution
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device is {device} ")

# define model parameters
NUM_EPOCHS = 1
BATCH_SIZE = 64
"""MOMENTUM = 0.9
LR_DECAY = 0.0005
LR_INIT = 0.01"""
IMAGE_DIM = 256  # pixels
NUM_CLASSES = 1  # classes
# modify this to point to your data directory
INPUT_ROOT_DIR = './red'
TRAIN_IMG_DIR = './red/train'
TEST_IMG_DIR = './red/test'
OUTPUT_DIR = './alexnet_data_out'
LOG_DIR = OUTPUT_DIR + '/tblogs'  # tensorboard logs
CHECKPOINT_DIR = OUTPUT_DIR + '/models'  # model checkpoints

# make checkpoint path directory
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

device is cuda 


In [ ]:
#train data
train_data = load_data(TRAIN_IMG_DIR, class_num = 2)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True)

#test data
test_data = load_data(TEST_IMG_DIR, class_num=2)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle = True)

print("train, test data loader created")

train, test data loader created


In [ ]:
train_size = len(train_data)
test_size = len(test_data)
print(train_size, test_size)
#print(train_data.type(), test_data.type())

11000 1000


In [ ]:
#Model
model = Model(class_num=NUM_CLASSES).to(device)
#print(alexnet)
print("lica model created...")

#Loss
criterion = nn.CrossEntropyLoss()
bce_loss = nn.BCELoss()

#Optimizer(Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

lica model created...


In [ ]:
#Training
print('Starting training...')
for epoch_idx in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_acc = 0
    for batch_data, batch_red, batch_label in train_loader:
        batch_label = batch_label.float()
        #batch_data = F.interpolate(batch_data, size=(227, 227), mode='bicubic', align_corners=False) # torch.Size([1, 3, 227, 227]) torch.float32
        batch_data, batch_red, batch_label = batch_data.to(device), batch_red.to(device), batch_label.to(device)
        #print(batch_label.type())

        #cal loss
        output = model(batch_data, batch_red)
        output = torch.squeeze(output, dim=1)
        loss = bce_loss(output, batch_label)


        #update the parameters
        optimizer.zero_grad()
        #loss = loss_data + loss_red
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        print(train_loss)
        train_pred = output >= torch.FloatTensor([0.5]).to(device)
        batch_bool = batch_label >= torch.FloatTensor([0.5]).to(device)
        train_acc += (train_pred == batch_bool).sum().item()
        #print(f"train_acc: {train_acc}")
    
    print(f"epoch : {epoch_idx+1} - train loss: {train_loss / (train_size / BATCH_SIZE)} , train acc: {train_acc / train_size}")

    #torch.save(LICA_alex.state_dict(), "./lica_alex_net.pth")
            

Starting training...
0.6700953245162964
1.2480003833770752
1.7609105110168457
2.204057812690735
2.6372478008270264
3.083079516887665
3.4125238060951233
3.735759288072586
4.053900748491287
4.227085590362549
4.389987200498581
4.594285547733307
4.758654743432999
4.866228066384792
5.00545122474432
5.183044977486134
5.345476426184177
5.417100615799427
5.490369409322739
5.560727298259735
5.617455825209618
5.660680271685123
5.733567424118519
5.7703867591917515
5.8074457086622715
5.817622811533511
5.901834548451006
5.967778765596449
5.988065687008202
6.006246958859265
6.01138088433072
6.02348301326856
6.055593346711248
6.110736919101328
6.1199789880774915
6.127214634325355
6.1313255545683205
6.133602250367403
6.144456060603261
6.165355522185564
6.167102585546672
6.17227094899863
6.187578680925071
6.188288883888163
6.1893891006475314
6.189894631272182
6.192780223907903
6.208918269025162
6.209174375369912
6.209546079655411
6.20960651346104
6.216875996055023
6.217225211403274
6.2176456652450725
6

In [ ]:
 #Test
model.eval()
test_loss = 0.0
test_accuracy = 0
print("test start")
with torch.no_grad():
    for test_data, test_red, test_label in test_loader:
        #test_data = F.interpolate(test_data, size=(227, 227), mode='bicubic', align_corners=False) # torch.Size([1, 3, 227, 227]) torch.float32
        test_label = test_label.float()
        test_data, test_red, test_label = test_data.to(device), test_red.to(device), test_label.to(device)

        #calculate the loss
        test_output = model(test_data,test_red)
        test_output = torch.squeeze(test_output, dim=1)
        loss = bce_loss(test_output, test_label)
            
        test_loss += loss.item()
        test_pred = test_output >= torch.FloatTensor([0.5]).to(device)
        batch_bool = test_label >= torch.FloatTensor([0.5]).to(device)
        test_accuracy += (test_pred == batch_bool).sum().item()

    print(f"test loss : {test_loss / (test_size / BATCH_SIZE)}, test accuracy : {test_accuracy / test_size}")

test start
test loss : 9.337477594755228e-07, test accuracy : 1.0


In [ ]:
torch.save(model.state_dict(), "./lica_color_model.pth")